In [289]:
from __future__ import absolute_import, division, print_function, unicode_literals

In [290]:
import tensorflow as tf

In [291]:
import numpy as np
import matplotlib as mpl
import IPython.display as display
import PIL.Image

from tensorflow.keras.preprocessing import image

import random
import datetime
import os
import pickle

In [292]:
def img_to_numpy(img_path, max_dim=None):
    image_path = img_path
    img = PIL.Image.open(image_path)
    if max_dim:
        img.thumbnail((max_dim, max_dim))
    return np.array(img)

# Normalize an image
def deprocess(img):
    img = 255*(img + 1.0)/2.0
    return tf.cast(img, tf.uint8)

# Display an image
def show(img):
    display.display(PIL.Image.fromarray(np.array(img)))


image_files = []
numpy_images = []

path, dirs, files = next(os.walk("./data/"))
file_count = len(files) - 1
print("images:", file_count)

# iterate through files in /data directory
for i in range(0, file_count):
    print('frame_' + str(i) + '.jpg')
    image_files.append(path +'frame_' + str(i) + '.jpg')
    
# convert those image files to numpy array
for file in image_files:
    numpy_images.append(img_to_numpy(file, max_dim=500))



images: 186
frame_0.jpg
frame_1.jpg
frame_2.jpg
frame_3.jpg
frame_4.jpg
frame_5.jpg
frame_6.jpg
frame_7.jpg
frame_8.jpg
frame_9.jpg
frame_10.jpg
frame_11.jpg
frame_12.jpg
frame_13.jpg
frame_14.jpg
frame_15.jpg
frame_16.jpg
frame_17.jpg
frame_18.jpg
frame_19.jpg
frame_20.jpg
frame_21.jpg
frame_22.jpg
frame_23.jpg
frame_24.jpg
frame_25.jpg
frame_26.jpg
frame_27.jpg
frame_28.jpg
frame_29.jpg
frame_30.jpg
frame_31.jpg
frame_32.jpg
frame_33.jpg
frame_34.jpg
frame_35.jpg
frame_36.jpg
frame_37.jpg
frame_38.jpg
frame_39.jpg
frame_40.jpg
frame_41.jpg
frame_42.jpg
frame_43.jpg
frame_44.jpg
frame_45.jpg
frame_46.jpg
frame_47.jpg
frame_48.jpg
frame_49.jpg
frame_50.jpg
frame_51.jpg
frame_52.jpg
frame_53.jpg
frame_54.jpg
frame_55.jpg
frame_56.jpg
frame_57.jpg
frame_58.jpg
frame_59.jpg
frame_60.jpg
frame_61.jpg
frame_62.jpg
frame_63.jpg
frame_64.jpg
frame_65.jpg
frame_66.jpg
frame_67.jpg
frame_68.jpg
frame_69.jpg
frame_70.jpg
frame_71.jpg
frame_72.jpg
frame_73.jpg
frame_74.jpg
frame_75.jpg
frame_76.j

In [293]:
base_model = tf.keras.applications.InceptionV3(include_top=False, weights='imagenet')

In [294]:
def calc_loss(img, model):
  # Pass forward the image through the model to retrieve the activations.
  # Converts the image into a batch of size 1.
    img_batch = tf.expand_dims(img, axis=0)
    layer_activations = model(img_batch)
    if len(layer_activations) == 1:
        layer_activations = [layer_activations]

    losses = []
    for act in layer_activations:
        loss = tf.math.reduce_mean(act)
        losses.append(loss)

    return  tf.reduce_sum(losses)

In [295]:
class DeepDream(tf.Module):
    def __init__(self, model):
        self.model = model

    @tf.function(
        input_signature=(
            tf.TensorSpec(shape=[None,None,3], dtype=tf.float32),
            tf.TensorSpec(shape=[], dtype=tf.int32),
            tf.TensorSpec(shape=[], dtype=tf.float32),)
    )
    
    def __call__(self, img, steps, step_size):
        print("Tracing")
        loss = tf.constant(0.0)
        for n in tf.range(steps):
            with tf.GradientTape() as tape:
              # This needs gradients relative to `img`
              # `GradientTape` only watches `tf.Variable`s by default
                tape.watch(img)
                loss = calc_loss(img, self.model)

            # Calculate the gradient of the loss with respect to the pixels of the input image.
            gradients = tape.gradient(loss, img)

            # Normalize the gradients.
            gradients /= tf.math.reduce_std(gradients) + 1e-8 
        
            # In gradient ascent, the "loss" is maximized so that the input image increasingly "excites" the layers.
            # You can update the image by directly adding the gradients (because they're the same shape!)
            img = img + gradients*step_size
            img = tf.clip_by_value(img, -1, 1)

        return loss, img

In [296]:
def random_roll(img, maxroll):
  # Randomly shift the image to avoid tiled boundaries.
    shift = tf.random.uniform(shape=[2], minval=-maxroll, maxval=maxroll, dtype=tf.int32)
    shift_down, shift_right = shift[0],shift[1] 
    img_rolled = tf.roll(tf.roll(img, shift_right, axis=1), shift_down, axis=0)
    return shift_down, shift_right, img_rolled



In [297]:
class TiledGradients(tf.Module):
    def __init__(self, model):
        self.model = model

    @tf.function(
        input_signature=(
            tf.TensorSpec(shape=[None,None,3], dtype=tf.float32),
            tf.TensorSpec(shape=[], dtype=tf.int32),)
    )
    
    def __call__(self, img, tile_size=512):
        shift_down, shift_right, img_rolled = random_roll(img, tile_size)

    # Initialize the image gradients to zero.
        gradients = tf.zeros_like(img_rolled)
    
    # Skip the last tile, unless there's only one tile.
        xs = tf.range(0, img_rolled.shape[0], tile_size)[:-1]
        if not tf.cast(len(xs), bool):
            xs = tf.constant([0])
        ys = tf.range(0, img_rolled.shape[1], tile_size)[:-1]
        if not tf.cast(len(ys), bool):
            ys = tf.constant([0])

        for x in xs:
            for y in ys:
            # Calculate the gradients for this tile.
                with tf.GradientTape() as tape:
                    # This needs gradients relative to `img_rolled`.
                    # `GradientTape` only watches `tf.Variable`s by default.
                    tape.watch(img_rolled)

                    # Extract a tile out of the image.
                    img_tile = img_rolled[x:x+tile_size, y:y+tile_size]
                    loss = calc_loss(img_tile, self.model)

            # Update the image gradients for this tile.
                gradients = gradients + tape.gradient(loss, img_rolled)

        # Undo the random shift applied to the image and its gradients.
        gradients = tf.roll(tf.roll(gradients, -shift_right, axis=1), -shift_down, axis=0)

        # Normalize the gradients.
        gradients /= tf.math.reduce_std(gradients) + 1e-8 

        return gradients 



In [298]:
def run_deep_dream_with_octaves(img, steps_per_octave=100, step_size=0.01, octaves=range(-3, -2), octave_scale=1.2):
    
    base_shape = tf.shape(img)
    img = tf.keras.preprocessing.image.img_to_array(img)
    img = tf.keras.applications.inception_v3.preprocess_input(img)

    initial_shape = img.shape[:-1]
    img = tf.image.resize(img, initial_shape)
    for octave in octaves:
        # Scale the image based on the octave
        new_size = tf.cast(tf.convert_to_tensor(base_shape[:-1]), tf.float32)*(octave_scale**octave)
        img = tf.image.resize(img, tf.cast(new_size, tf.int32))

        for step in range(steps_per_octave):
            gradients = get_tiled_gradients(img)
            img = img + gradients*step_size
            img = tf.clip_by_value(img, -1, 1)

            if step % 10 == 0:
                display.clear_output(wait=True)
                show(deprocess(img))
                print ("Octave {}, Step {}".format(octave, step))
    
    result = deprocess(img)
    base_shape = tf.shape(result)[:-1]

    resize_img = tf.image.resize(result, base_shape)
    dtype_img = tf.image.convert_image_dtype(resize_img/255.0, dtype=tf.uint8)
    # show(result)
    
    np_image = np.array(dtype_img)
    final_image = PIL.Image.fromarray(np_image)

    return final_image



In [299]:
# function to translate values from one number range to a new range
def translate(value, leftMin, leftMax, rightMin, rightMax):
    # Figure out how 'wide' each range is
    leftSpan = leftMax - leftMin
    rightSpan = rightMax - rightMin

    # Convert the left range into a 0-1 range (float)
    valueScaled = float(value - leftMin) / float(leftSpan)

    # Convert the 0-1 range into a value in the right range.
    return round(rightMin + (valueScaled * rightSpan))



In [300]:
names = ['mixed0', 'mixed1', 'mixed2', 'mixed3', 'mixed4', 'mixed5', 'mixed6', 'mixed7', 'mixed8', 'mixed9', 'mixed10']
layers = [base_model.get_layer(name).output for name in names]
             
layer_number = 0
image_number = 0
min = 0


In [301]:
pickle_exists = os.path.isfile('save_state.pickle')

if pickle_exists == True:
    
    print("loading pickle...")
    with open("save_state.pickle", 'rb') as pickle_file:
        layer_number = pickle.load(pickle_file)
        min = pickle.load(pickle_file) 
        
else:
    
    print("creating new pickle...")
    with open("save_state.pickle", "wb") as pickle_file:
        pickle.dump(layer_number, pickle_file)
        pickle.dump(min, pickle_file)

loading pickle...


In [1]:
for i in range(min, len(numpy_images)):
    
        min = i
        print('frame' + str(min) + '.jpg')
    
        # map current index number to a range between 8, 0 for our convolutional layer
        layer_number = translate(min, 0, len(numpy_images) - 1, 5, 6)
        print("layer_number", layer_number)

        # load models and select layers 
        dream_model = tf.keras.Model(inputs=base_model.input, outputs=layers[layer_number])
        deepdream = DeepDream(dream_model)
        get_tiled_gradients = TiledGradients(dream_model)

        # save dream image to dream_data directory
        dream_img = run_deep_dream_with_octaves(img=numpy_images[min], step_size=0.01)
        dream_img.save('./dream_data/dream_frame_' + str(i) + '.png')    
    
        with open("save_state.pickle", "wb") as pickle_file:
            pickle.dump(layer_number, pickle_file)
            pickle.dump(min, pickle_file)
            
  
print('done!')



NameError: name 'numpy_images' is not defined